# Importações

In [1]:
import numpy as np
from PIL import Image
import glob
import matplotlib.pyplot as plt
from sklearn.utils.extmath import randomized_svd
import utils_imagem as utl
import sys
import locale

# Definição de constantes

In [2]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

m = 180 # altura da imagem
n = 130 # largura da imagem
DIMENSAO_IMAGEM = (m, n)

# Altera o valor da constante no módulo de processamento de imagens
utl.DIMENSAO_IMAGEM = DIMENSAO_IMAGEM

# Gerar matrizes A, M, C, U_A, U_C e a face média 

In [3]:
# Matriz A(mn x q) na qual as colunas são as 'q' imagens do banco de imagens
(q, A) = utl.varrer_banco_imagens('src/FEI_face_db/*')

# Vetor f(mn x 1) com a média das colunas A, no caso, a face média
face_media = np.mean(A, axis=1, keepdims=True) 

# Matriz M(mn x q) de média corrigida
M = A - face_media

In [4]:
A

array([[0.78431373, 0.74509804, 0.84313725, ..., 1.        , 0.72941176,
        0.72941176],
       [0.78431373, 0.74509804, 0.84705882, ..., 1.        , 0.73333333,
        0.7254902 ],
       [0.78431373, 0.74117647, 0.85098039, ..., 1.        , 0.73333333,
        0.73333333],
       ...,
       [0.29019608, 0.44313725, 0.05490196, ..., 0.02352941, 0.45882353,
        0.5254902 ],
       [0.30196078, 0.45098039, 0.0627451 , ..., 0.02352941, 0.45490196,
        0.5254902 ],
       [0.29803922, 0.45882353, 0.05882353, ..., 0.01960784, 0.45882353,
        0.52156863]])

In [5]:

 
# memory size of numpy array
print(f"A ocupa: {(A.size * A.itemsize) / (1024.0 ** 2):,} Mb")

A ocupa: 71.4111328125 Mb


In [16]:
# Matriz de covariância de A, C(mn x mn)
C = np.cov(M)

In [8]:
# Decomposição SVD na forma reduzida por causa de 'full_matrices=False'
U_A, autovalores_A, _ = np.linalg.svd(A, full_matrices=False) # U já está ordenada em relação aos autovalores

In [9]:
# Decomposição SVD na forma reduzida por causa de 'full_matrices=False'
componentes = U_A.shape[1] # valor arbitrário, decidido para as matrizes ficarem pareadas
U_C, autovalores_C, VT_C = randomized_svd(C, n_components=componentes, n_iter=5, random_state=None)

# Verifica tamanho de variáveis

In [10]:
qtd_pixels = f'{m*n:,}'
print(f'Foram carregadas {q} imagens, cada uma com {qtd_pixels.replace(",",".")} pixels')

Foram carregadas 400 imagens, cada uma com 23.400 pixels


In [11]:
# memory size of numpy array
print(f"A ocupa: {(A.size * A.itemsize) / (1024.0 ** 2):,} Mb")

A ocupa: 71.4111328125 Mb


In [61]:
lista_nomes = ['  A', '  M', '  C', 'U_A', 'U_C']
lista_var = (A, M, C, U_A, U_C)
lista_magnitude = ['mb', 'gb']
    
for i in range(len(lista_nomes)):
    ocupacao = ''
    
    for j in range(2):
        tamanho = f'{(lista_var[i].size * lista_var[i].itemsize) / (1024.0 ** (j+2)):_.1f}'.replace(".",",").replace("_",".")
        ocupacao = ocupacao + f'{tamanho} {lista_magnitude[j]} = '

    print(f'{lista_nomes[i]} ocupa {ocupacao[:-2]}')

  A ocupa 71,4 mb = 0,1 gb 
  M ocupa 71,4 mb = 0,1 gb 
  C ocupa 4.177,6 mb = 4,1 gb 
U_A ocupa 71,4 mb = 0,1 gb 
U_C ocupa 71,4 mb = 0,1 gb 


In [13]:
# Apaga C da memória (liberando 4gb) por não ser mais necessária
del C